In [108]:
import random
import math

debug=False #if(debug)
debug=True

class crypto:
    P = 2 ** 256 - 2 ** 32 - 2 ** 9 - 2 ** 8 - 2 ** 7 - 2 ** 6 - 2 ** 4 - 1  # prime number for modulus operations
    order = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141  # order for the elliptic curve y^2=x^3+7 ,used in bitcoin
    Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240  # x co-ordinate of generating point of secp256k1 i.e. curve used in bitcoin
    Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424  # y co-ordinate of generating point of secp256k1 i.e. curve used in bitcoin

    # mod inverse
    # Extended Euclidean Algorithm for finding mod inverse
    def modinv(self,a, n):
        lm, hm = 1, 0
        low, high = a % n, n
        while low > 1:
            ratio = high // low
            nm = hm - lm * ratio
            new = high - low * ratio
            hm = lm
            high = low
            lm = nm
            low = new
        return lm % n

    # Elliptic curve point addition
    def ECadd(self,xp, yp, xq, yq):
        # point addition will not work if both the point are same. So, point doubling is required
        if xp == xq and yp == yq:
            return self.ECdouble(xp, yp)

        m = ((yq - yp) * self.modinv(xq - xp, self.P)) % self.P
        xr = (m * m - xp - xq) % self.P
        yr = (m * (xp - xr) - yp) % self.P
        return (xr, yr)

    # Elliptic curve point doubling
    def ECdouble(self,xp, yp):
        l = ((3 * xp * xp) * self.modinv(2 * yp, self.P)) % self.P
        X = (l * l - 2 * xp) % self.P
        Y = (l * (xp - X) - yp) % self.P
        return X, Y

    # Elliptic curve point multiplication
    ##1. scaler multiplication with generator point
    def ECmult(self,scaler):
        if scaler == 0:
            return 0, 0
        _2pX = [0] * 258
        _2pY = [0] * 258
        _2pX[0], _2pY[0] = self.Gx, self.Gy
        _X = self.Gx
        _Y = self.Gy
        for i in range(1, 257):
            _2pX[i], _2pY[i] = self.ECdouble(_2pX[i - 1], _2pY[i - 1])

        index = 0
        while not (scaler & 1):
            index += 1
            scaler >>= 1
        _X = _2pX[index]
        _Y = _2pY[index]
        scaler >>= 1
        index += 1
        while scaler > 0:
            if scaler & 1:
                _X, _Y = self.ECadd(_X, _Y, _2pX[index], _2pY[index])
            scaler >>= 1
            index += 1
        return _X, _Y

    # fast EC scaler multiplication 4x faster than previous method
    # this can be improved more by catching precalculating group by few bits.
    def ECmult_fast(scaler):
        if scaler==0:
            return 0,0
        if(scaler==1):
            return Gx
        scaler-=1
        _2px=Gx
        _2py=Gy
        _x=Gx
        _y=Gy
        while(scaler):
            if scaler&1:
                _x,_y=ECadd(_x,_y,_2px,_2py)
            scaler>>=1
            _2px,_2py=ECdouble(_2px,_2py)
        return _x,_y

    ##2. scaler multiplication with other point on secp256k1 curve
    ###Example 2*(4G)=8G 4*(5G)=20G etc.
    def ECmultp(self,Sx, Sy, scaler):
        _2pX = [0] * 258
        _2pY = [0] * 258
        _2pX[0], _2pY[0] = Sx, Sy
        _X = Sx
        _Y = Sy
        for i in range(1, 257):
            _2pX[i], _2pY[i] = self.ECdouble(_2pX[i - 1], _2pY[i - 1])

        index = 0
        while not (scaler & 1):
            index += 1
            scaler >>= 1
        _X = _2pX[index]
        _Y = _2pY[index]
        scaler >>= 1
        index += 1

        while scaler > 0:
            if scaler & 1:
                _X, _Y = self.ECadd(_X, _Y, _2pX[index], _2pY[index])
            scaler >>= 1
            index += 1
        return _X, _Y

    # ECscalerDivide divide a ECpoint by a scaler number
    # e.g. 8G/2=4G
    def ECscalerDivide(self,x,y,scaler):
      return self.ECmultp(x,y,self.modinv(scaler,self.order))

    def ECxToyNotWorking(self,x):
      a = (x**3 + 7)%self.P
      ##_1_2 = self.modinv(2,self.order)
      y = self.tonelli_shanks(a,self.P)#pow(a,_1_2,self.P)%self.P
      if(y==None):
        return 0,0
      return y,(self.P-y)%self.P

    # find possible y coordinates w.r.t x coordinate
    # y^2 = x^3+7
    # y = pow_mod(x^3+7,P+1//4)
    def ECxToy(self,x):
      a = (pow(x, 3, self.P) + 7) % self.P
      if(self.tonelli_shanks(a,self.P)==None):
        print("sqrt doesnot exists") if debug else None
      y = pow(a, (self.P+1)//4, self.P)
      return y,(self.P-y)%self.P


    #Generate a private key
    def generate_private_key(self):
        st = 1 << 254
        en = self.order - 1
        rn = random.randrange(st, en)
        (rx, ry) = self.ECmult(rn)
        rn=(rx+ry)%self.order
        (rx, ry) = self.ECmult(rn)
        return ry;

    #private key to public key conversion i.e. x coordinate of G*priv
    def private_to_public(self,priv_k):
        return self.ECmult(priv_k)

    ## Signing message and returning random part and signature
    def sign_message(self,msg,rn,prv_k):
        print("signing....")
        #order=115792089237316195423570985008687907852837564279074904382605163141518161494337
        st=1<<254
        en=self.order-1
        #rn=random.randrange(st, en)
        #print(rn)
        (rx,ry)=self.ECmult(rn)
        #print(rx,ry)
        sign=(((rx*prv_k)%self.order+msg)*self.modinv(rn,self.order))%self.order
        print("rx:",rx)
        print("sign:",sign)
        return (rx,sign)

    #verifying message
    def verify_message(self,s,r,msg,pub_x,pub_y):
        print("verifying...")
        m_s=(msg*self.modinv(s,self.order))%self.order
        print("m_s:",m_s)
        r_s=(r*self.modinv(s,self.order))%self.order
        print("r_s:",r_s)
        (x,y)=self.ECmult(m_s)
        (x2,y2)=self.ECmultp(pub_x,pub_y,r_s)
        (x3,y3)=self.ECadd(x,y,x2,y2)
        print("(x,y):",(x,y))
        print("(x2,y2):",(x2,y2))
        print("(x3,y3):",(x3,y3))
        if(x3==r):
            #print("valid")
            return True
        else:
            #print("invalid")
            return False

    #encryption of message to be send
    #Currently this is only xor(NOT SECURE), it is to be changed to ECadd & ECsub
    def encrypt_message(self,msg,r,prv_k,pub_k):
        (px,py)=self.ECmultp(pub_k[0],pub_k[1],prv_k)
        (rx,ry)=self.ECmultp(px,py,r)
        enc_msg=0
        l=0
        cnt=0
        for ch in msg:
            cnt+=1
            # print(ord(ch))
            l<<=8
            l+=ord(ch)
            if(cnt==30):
                cnt=0
                l=l^rx
                enc_msg <<= 256
                enc_msg|=l
                l=0
        if(l):
            l = l ^ rx
            enc_msg <<= 256
            enc_msg |= l

        # print(l)
        # enc_msg=l+rx
        # print(enc_msg)
        return enc_msg
        # encoded_string = msg.encode()
        # byte_array = bytearray(encoded_string)
        # print(byte_array)


    # decryption of the received message
    # Currently this is only xor(NOT SECURE), it is to be changed to ECadd & ECsub Concept
    def decrypt_message(self,enc_msg,r,prv_k,pub_k):
        (px, py) = self.ECmultp(pub_k[0], pub_k[1], prv_k)
        (rx, ry) = self.ECmultp(px, py, r)
        msg=""
        while(enc_msg):
            l=enc_msg&((1<<256)-1)^rx
            while(l):
                ch_i=l&255
                msg=chr(ch_i)+msg
                l>>=8
            enc_msg>>=256
        #print(msg)
        return msg

    # Elliptic curve point subtraction
    def ECsubtract(self,p1x,p1y,p2x,p2y):
      if(p1x==p2x):
        print("invalid point subtraction")
        return (0,0)
      return self.ECadd(p1x,p1y,p2x,(self.P-p2y)%self.P)

    def isValidPoint(self,x,y):
      x3_7 = ((x**3)+7)%self.P
      y2 = (y**2)%self.P
      return x3_7==y2

    def legendre_symbol(self,a, p):
      ls = pow(a, (p - 1) // 2, p)
      return -1 if ls == p - 1 else ls

    def tonelli_shanks(self,n, p):
      if self.legendre_symbol(n, p) != 1:
          return None  # No square root exists

      q = p - 1
      s = 0
      while q % 2 == 0:
          q //= 2
          s += 1

      if s == 1:
          return pow(n, (p + 1) // 4, p)

      for z in range(2, p):
          if self.legendre_symbol(z, p) == -1:
              break

      c = pow(z, q, p)
      r = pow(n, (q + 1) // 2, p)
      t = pow(n, q, p)
      m = s

      while True:
          if t == 0:
              return 0
          if t == 1:
              return r
          i = 0
          zz = t
          while zz != 1 and i < m:
              zz = (zz * zz) % p
              i += 1
          b = pow(c, 1 << (m - i - 1), p)
          r = (r * b) % p
          t = (t * b * b) % p
          c = (b * b) % p
          m = i


    # encryption of message using point addition and substraction
    # Encrypted format: {point,encrypted_data}
    # algo:
    #   Generate random integer, k
    #   Multiply with G, P1 = K.G
    #   Encrypting message
    #        read 255bits or less from the message Mx
    #        generate encrypted msg i.e. Emx = Mx+k.receiver_pubk, append 0 if y is odd and 1 otherwise
    #        keep Appending 257bits Emx in the encrypted message

    # Encrypting every 31bytes generates 33bytes encrypted data, which is ~6% increase in size.
    # Can be done with 255bits to 257bits which is ~0.7% increase in size.

    def encrypt2_0(self,msg,receiver_pubk,isUsingFirst):
      print("\n################ EnCrypt ###################")
      bytes_value = msg.encode('utf-8')
      ln=len(bytes_value)
      n_blocks=math.ceil(ln/31.0)
      print(ln,n_blocks,bytes_value)

      enc_k = self.generate_private_key()#a random number k
      enc_p1 = self.private_to_public(enc_k)# k.G

      total_encrypted_msg = b''

      for i in range(n_blocks):
        msg=bytes_value[i*31:i*31+31]
        msgint=int.from_bytes(msg, byteorder='big')
        print(i," block:",msg)
        print("msgint",msgint)

        msgys = self.ECxToy(msgint)
        msgys2 = self.ECxToyNotWorking(msgint)
        # choose y as the odd one
        msgy_choosen = msgys[0]#default
        if(isUsingFirst != 1):
           msgy_choosen = msgys[1]

        # msgy_odd = msgys[0]
        # if((msgy_odd & 1)!= 1):
        #   msgy_odd = msgys[1]
        #print("msgys2",msgys2)
        print("msgys:",msgys," choosen:",msgy_choosen)
        print("validity:",self.isValidPoint(msgint,msgys[0]))
        print("validity:",self.isValidPoint(msgint,msgys[1]))
        print("validity 2:",self.isValidPoint(msgint,msgys2[0]))
        print("validity 2:",self.isValidPoint(msgint,msgys2[1]))

        # creating k.Pb
        kPb = self.ECmultp(receiver_pubk[0],receiver_pubk[1],enc_k)
        print("k.Pb:",kPb)

        # add kpb with msg
        msg_kPb = self.ECadd(msgint,msgy_choosen,kPb[0],kPb[1])
        print("msg_kPb:",msg_kPb)

        enc_msg_chunk  = msg_kPb[0].to_bytes(32, byteorder='big')
        print("enc_msg_chunk:",enc_msg_chunk)

        if (msg_kPb[1]&1 == 1):
          enc_msg_chunk = b'1'+enc_msg_chunk
        else:
          enc_msg_chunk = b'0'+enc_msg_chunk

        print(len(enc_msg_chunk),"enc_msg_chunk:",enc_msg_chunk)
        total_encrypted_msg = total_encrypted_msg + enc_msg_chunk

      return (enc_p1,total_encrypted_msg)

    def decrypt2_0(self,enc_points,receiver_privk):
      print("\n################ DeCrypt ###################")
      enc_p1 = enc_points[0]
      enc_msg = enc_points[1]
      print("enc_p1:",enc_p1)
      print("enc_msg:",enc_msg)

      # k.G.priv
      kGprvk = self.ECmultp(enc_p1[0],enc_p1[1],receiver_privk)
      print("kGprvk:",kGprvk)

      total_decrypted_str = ""

      #iterate through all blocks
      ln=len(enc_msg)
      n_blocks=math.ceil(ln/33.0)
      for i in range(n_blocks):
        enc_msg_block=enc_msg[i*33:i*33+33]
        y_sign=enc_msg_block[0:1]
        enc_msg_block=enc_msg_block[1:]
        print("y_sign",y_sign,"enc_msg_block:",enc_msg_block)

        y_sign_int=int.from_bytes(y_sign, byteorder='big')
        enc_msg_block_int=int.from_bytes(enc_msg_block, byteorder='big')
        print("y_sign_int",y_sign_int,"enc_msg_block_int:",enc_msg_block_int)

        # find y for enc_msg_block_int to form the point
        ys_for_enc_msg_block_int = self.ECxToy(enc_msg_block_int)
        print("ys_for_enc_msg_block_int:",ys_for_enc_msg_block_int)
        y_for_enc_msg_block_int=ys_for_enc_msg_block_int[0]

        if(((y_for_enc_msg_block_int&1)^(y_sign_int&1))!=0):
          y_for_enc_msg_block_int=ys_for_enc_msg_block_int[1]
        print("y_for_enc_msg_block_int:",y_for_enc_msg_block_int)

        # decrypt the message
        dec_msg_int = self.ECsubtract(enc_msg_block_int,y_for_enc_msg_block_int,kGprvk[0],kGprvk[1])
        print(dec_msg_int)
        msg_bytes_value = dec_msg_int[0].to_bytes(32, byteorder='big')
        print("msg_bytes_value:",msg_bytes_value)

        null_byte_str = b'\x00'.decode('utf-8')
        string_value = msg_bytes_value.decode('utf-8', errors='replace')
        string_value = string_value.lstrip(null_byte_str)
        print("string_value:",string_value)
        total_decrypted_str = total_decrypted_str + string_value
      return total_decrypted_str


    check_overhead_count_map = {}
    def add_in_check_overhead_count_map(self,num):
      if(num in self.check_overhead_count_map):
        self.check_overhead_count_map[num]+=1
      else:
        self.check_overhead_count_map[num]=1
    def print_check_overhead_count_map(self):
      sorted_count_map = dict(sorted(self.check_overhead_count_map.items()))
      for num, count in sorted_count_map.items():
        print(f"{num}: {count}")

    def createValidMsg(self,msg_bytes):#prefix_to_make_valid_point
      msgint=int.from_bytes(msg_bytes, byteorder='big')
      print("msgint",msgint) if debug else None
      msgys = self.ECxToy(msgint)
      if(self.isValidPoint(msgint,msgys[0]) and self.isValidPoint(msgint,msgys[1])):
        self.add_in_check_overhead_count_map(0)
        return False,msg_bytes

      for i in range(1,128):
        padded_msg_bytes = i.to_bytes(1, byteorder='big')+msg_bytes
        msgint=int.from_bytes(padded_msg_bytes, byteorder='big')
        print("padded_msg_bytes",padded_msg_bytes) if debug else None
        print("msgint",msgint)  if debug else None
        msgys = self.ECxToy(msgint)
        if(self.isValidPoint(msgint,msgys[0]) and self.isValidPoint(msgint,msgys[1])):
          print("### valid message found at ",i," ###")  if debug else None
          self.add_in_check_overhead_count_map(i)
          return True,padded_msg_bytes

      self.add_in_check_overhead_count_map(256)
      return None,None

    def encrypt3_0(self,msg,receiver_pubk,isUsingFirst):
      print("\n################ EnCrypt ###################") if debug else None
      bytes_value = msg.encode('utf-8')
      ln=len(bytes_value)
      n_blocks=math.ceil(ln/31.0)
      print(ln,n_blocks,bytes_value)

      enc_k = self.generate_private_key()#a random number k
      enc_p1 = self.private_to_public(enc_k)# k.G

      print("enc_p1 validity:",self.isValidPoint(enc_p1[0],enc_p1[1]))

      total_encrypted_msg = b''

      for i in range(n_blocks):
        print("\n",i," block:") if debug else None
        msg=bytes_value[i*31:i*31+31]
        isAddedByte,msg = self.createValidMsg(msg)
        if(msg == None):
          print("############# cannot find valid msg ############")
          return ((0,0),0)
        msgint=int.from_bytes(msg, byteorder='big')
        print(" msg:",msg) if debug else None
        print("msgint",msgint) if debug else None

        msgys = self.ECxToy(msgint)
        msgys2 = self.ECxToyNotWorking(msgint)
        # choose y as the odd one
        msgy_choosen = msgys[0]#default
        if(isUsingFirst != 1):
           msgy_choosen = msgys[1]


        # msgy_odd = msgys[0]
        # if((msgy_odd & 1)!= 1):
        #   msgy_odd = msgys[1]
        #print("msgys2",msgys2)
        print("msgys:",msgys," choosen:",msgy_choosen) if debug else None
        print("validity:",self.isValidPoint(msgint,msgys[0])) if debug else None
        print("validity:",self.isValidPoint(msgint,msgys[1])) if debug else None
        print("validity 2:",self.isValidPoint(msgint,msgys2[0])) if debug else None
        print("validity 2:",self.isValidPoint(msgint,msgys2[1])) if debug else None

        # creating k.Pb
        kPb = self.ECmultp(receiver_pubk[0],receiver_pubk[1],enc_k)
        print("k.Pb:",kPb) if debug else None

        # add kpb with msg
        msg_kPb = self.ECadd(msgint,msgy_choosen,kPb[0],kPb[1])
        print("msg_kPb:",msg_kPb) if debug else None

        enc_msg_chunk  = msg_kPb[0].to_bytes(32, byteorder='big')
        print("enc_msg_chunk:",enc_msg_chunk) if debug else None

        info_byte = 0
        if (msg_kPb[1]&1 == 1):
          info_byte = info_byte | 1

        if(isAddedByte == True):
          info_byte = info_byte | 2
        print("info byte:",info_byte,"isAddedByte:",isAddedByte,"lsb msg_kPb[1]",msg_kPb[1]&1) if debug else None

        enc_msg_chunk = info_byte.to_bytes(1, byteorder='big')+enc_msg_chunk

        print(len(enc_msg_chunk),"enc_msg_chunk:",enc_msg_chunk) if debug else None
        total_encrypted_msg = total_encrypted_msg + enc_msg_chunk

      return (enc_p1,total_encrypted_msg)

    def decrypt3_0(self,enc_points,receiver_privk):
      print("\n################ DeCrypt ###################") if debug else None
      enc_p1 = enc_points[0]
      enc_msg = enc_points[1]
      print("enc_p1:",enc_p1) if debug else None
      print("enc_msg:",enc_msg) if debug else None

      # k.G.priv
      kGprvk = self.ECmultp(enc_p1[0],enc_p1[1],receiver_privk)
      print("kGprvk:",kGprvk) if debug else None

      total_decrypted_str = ""

      #iterate through all blocks
      ln=len(enc_msg)
      n_blocks=math.ceil(ln/33.0)
      for i in range(n_blocks):
        print("\n",i," block ") if debug else None
        enc_msg_block=enc_msg[i*33:i*33+33]
        info_byte=enc_msg_block[0:1]
        enc_msg_block=enc_msg_block[1:]
        print("info_byte",info_byte,"enc_msg_block:",enc_msg_block) if debug else None

        info_byte_int=int.from_bytes(info_byte, byteorder='big')
        enc_msg_block_int=int.from_bytes(enc_msg_block, byteorder='big')
        print("info_byte_int",info_byte_int,"enc_msg_block_int:",enc_msg_block_int) if debug else None

        # find y for enc_msg_block_int to form the point
        ys_for_enc_msg_block_int = self.ECxToy(enc_msg_block_int)
        print("ys_for_enc_msg_block_int:",ys_for_enc_msg_block_int) if debug else None
        y_for_enc_msg_block_int=ys_for_enc_msg_block_int[0]

        if(((y_for_enc_msg_block_int&1)^(info_byte_int&1))!=0):
          y_for_enc_msg_block_int=ys_for_enc_msg_block_int[1]
        print("y_for_enc_msg_block_int:",y_for_enc_msg_block_int) if debug else None

        # decrypt the message
        dec_msg_int = self.ECsubtract(enc_msg_block_int,y_for_enc_msg_block_int,kGprvk[0],kGprvk[1])
        print(dec_msg_int) if debug else None
        msg_bytes_value = dec_msg_int[0].to_bytes(32, byteorder='big')
        print("msg_bytes_value:",msg_bytes_value) if debug else None

        null_byte_str = b'\x00'.decode('utf-8')
        string_value = msg_bytes_value.decode('utf-8', errors='replace')
        string_value = string_value.lstrip(null_byte_str)
        print("string_value:",string_value) if debug else None
        if(info_byte_int&2 == 2):
          string_value = string_value[1:]
          print("wp string_value:",string_value) if debug else None



        total_decrypted_str = total_decrypted_str + string_value

      return total_decrypted_str

    def encrypt3_1(self,msg,receiver_pubk,isUsingFirst):
      print("\n################ EnCrypt ###################") if debug else None
      bytes_value = msg.encode('utf-8')
      ln=len(bytes_value)
      n_blocks=math.ceil(ln/31.0)
      print(ln,n_blocks,bytes_value)#if debug else None

      enc_k = self.generate_private_key()#a random number k
      enc_p1 = self.private_to_public(enc_k)# k.G

      print("enc_p1:",enc_p1)if debug else None
      print("enc_p1 validity:",self.isValidPoint(enc_p1[0],enc_p1[1]))if debug else None

      total_encrypted_msg = b''

      for i in range(n_blocks):
        print("\n",i," block:") if debug else None
        msg=bytes_value[i*31:i*31+31]
        isAddedByte,msg = self.createValidMsg(msg)
        if(msg == None):
          print("############# cannot find valid msg ############")
          return None
        msgint=int.from_bytes(msg, byteorder='big')
        print(" msg:",msg) if debug else None
        print("msgint",msgint) if debug else None

        msgys = self.ECxToy(msgint)
        msgys2 = self.ECxToyNotWorking(msgint)
        # choose y as the odd one
        msgy_choosen = msgys[0]#default
        if(isUsingFirst != 1):
           msgy_choosen = msgys[1]


        # msgy_odd = msgys[0]
        # if((msgy_odd & 1)!= 1):
        #   msgy_odd = msgys[1]
        #print("msgys2",msgys2)
        print("msgys:",msgys," choosen:",msgy_choosen) if debug else None
        print("validity:",self.isValidPoint(msgint,msgys[0])) if debug else None
        print("validity:",self.isValidPoint(msgint,msgys[1])) if debug else None
        print("validity 2:",self.isValidPoint(msgint,msgys2[0])) if debug else None
        print("validity 2:",self.isValidPoint(msgint,msgys2[1])) if debug else None

        # creating k.Pb
        kPb = self.ECmultp(receiver_pubk[0],receiver_pubk[1],enc_k)
        print("k.Pb:",kPb) if debug else None

        # add kpb with msg
        msg_kPb = self.ECadd(msgint,msgy_choosen,kPb[0],kPb[1])
        print("msg_kPb:",msg_kPb) if debug else None

        enc_msg_chunk  = msg_kPb[0].to_bytes(32, byteorder='big')
        print("enc_msg_chunk:",enc_msg_chunk) if debug else None

        info_byte = 0
        if (msg_kPb[1]&1 == 1):
          info_byte = info_byte | 1

        if(isAddedByte == True):
          info_byte = info_byte | 2
        print("info byte:",info_byte,"isAddedByte:",isAddedByte,"lsb msg_kPb[1]",msg_kPb[1]&1) if debug else None

        enc_msg_chunk = info_byte.to_bytes(1, byteorder='big')+enc_msg_chunk

        print(len(enc_msg_chunk),"enc_msg_chunk:",enc_msg_chunk) if debug else None
        total_encrypted_msg = total_encrypted_msg + enc_msg_chunk


      encPointX = enc_p1[0].to_bytes(32,byteorder='big')
      encPointY = 0
      if(enc_p1[1]&1):
        encPointY = 1
      encPointY = encPointY.to_bytes(1,byteorder='big')
      encPoint=encPointY+encPointX


      total_encrypted_msg = encPoint + total_encrypted_msg
      return total_encrypted_msg

    def decrypt3_1(self,total_encrypted_msg,receiver_privk):
      print("\n################ DeCrypt ###################") if debug else None

      # creating encryption point
      enc_pointXbyte = total_encrypted_msg[1:33]
      enc_pointYbyteInfo = total_encrypted_msg[0:1]
      enc_pointXint = int.from_bytes(enc_pointXbyte, byteorder='big')
      enc_pointYintInfo = int.from_bytes(enc_pointYbyteInfo, byteorder='big')
      enc_pointYs = self.ECxToy(enc_pointXint)

      enc_pointYint = enc_pointYs[0]
      if((enc_pointYintInfo&1)!=(enc_pointYint&1)):
        enc_pointYint = enc_pointYs[1]

      enc_p1 = (enc_pointXint,enc_pointYint)
      enc_msg = total_encrypted_msg[33:]
      print("enc_p1:",enc_p1) if debug else None
      print("enc_msg:",enc_msg) if debug else None

      # k.G.priv
      kGprvk = self.ECmultp(enc_p1[0],enc_p1[1],receiver_privk)
      print("kGprvk:",kGprvk) if debug else None

      total_decrypted_str = ""

      #iterate through all blocks
      ln=len(enc_msg)
      n_blocks=math.ceil(ln/33.0)
      for i in range(n_blocks):
        print("\n",i," block ") if debug else None
        enc_msg_block=enc_msg[i*33:i*33+33]
        info_byte=enc_msg_block[0:1]
        enc_msg_block=enc_msg_block[1:]
        print("info_byte",info_byte,"enc_msg_block:",enc_msg_block) if debug else None

        info_byte_int=int.from_bytes(info_byte, byteorder='big')
        enc_msg_block_int=int.from_bytes(enc_msg_block, byteorder='big')
        print("info_byte_int",info_byte_int,"enc_msg_block_int:",enc_msg_block_int) if debug else None

        # find y for enc_msg_block_int to form the point
        ys_for_enc_msg_block_int = self.ECxToy(enc_msg_block_int)
        print("ys_for_enc_msg_block_int:",ys_for_enc_msg_block_int) if debug else None
        y_for_enc_msg_block_int=ys_for_enc_msg_block_int[0]

        if(((y_for_enc_msg_block_int&1)^(info_byte_int&1))!=0):
          y_for_enc_msg_block_int=ys_for_enc_msg_block_int[1]
        print("y_for_enc_msg_block_int:",y_for_enc_msg_block_int) if debug else None

        # decrypt the message
        dec_msg_int = self.ECsubtract(enc_msg_block_int,y_for_enc_msg_block_int,kGprvk[0],kGprvk[1])
        print(dec_msg_int) if debug else None
        msg_bytes_value = dec_msg_int[0].to_bytes(32, byteorder='big')
        print("msg_bytes_value:",msg_bytes_value) if debug else None

        null_byte_str = b'\x00'.decode('utf-8')
        string_value = msg_bytes_value.decode('utf-8', errors='replace')
        string_value = string_value.lstrip(null_byte_str)
        print("string_value:",string_value) if debug else None
        if(info_byte_int&2 == 2):
          string_value = string_value[1:]
          print("wp string_value:",string_value) if debug else None



        total_decrypted_str = total_decrypted_str + string_value

      return total_decrypted_str


In [122]:
# 18 April 2024
# testing encryption/decryption 3.1

import random
import string
debug = False

def generate_random_word(length):
  letters = string.ascii_lowercase
  #letters = string.printable.strip()
  return ''.join(random.choice(letters) for _ in range(length))

def generate_random_sentence(min_n_words, max_n_words, min_length, max_length):
  num_words = random.randint(min_n_words, max_n_words)
  return ' '.join(generate_random_word(random.randint(min_length, max_length)) for _ in range(num_words))

cr = crypto()

pk = cr.generate_private_key()
pubk = cr.private_to_public(pk)
print(pubk)
print(pubk[0])
cnt=0
tcnt=0
n = 20
for i in range(n):
  print("(",i+1,"/",n,"):\t",round(((i+1)/n)*100),"% :\t",end=" ")
  message_to_encrypt = generate_random_sentence(500,1000,5,8) #"hello world hello! hello world hello! hello world hello! hello world hello! hello world hello! hello world hello!"
  encrypted_points = cr.encrypt3_1(message_to_encrypt,pubk,1)
  decrypted_msg = cr.decrypt3_1(encrypted_points,pk)
  tcnt+=1
  if(decrypted_msg==message_to_encrypt):
    cnt+=1
print("\nPassed:",cnt," of ",tcnt)

print("\npadding_overhead_count_map:")
cr.print_check_overhead_count_map()



# print("\n\nencrypted:",encrypted_points)if debug else None
# print("\n\n########################in transit ####################\n",encrypted_points)if debug else None

# print("\n\ndecrypted:",decrypted_msg)if debug else None

# print("\ncompare:before :",message_to_encrypt)if debug else None
# print("compare:after  :",decrypted_msg)if debug else None
# print("compare:isEqual:",decrypted_msg==message_to_encrypt)if debug else None


(107446742780430176674028494327449923011041703991733877197660295437827363809989, 111628151532609922731045190308355378447022725489977235568064434396658518939256)
107446742780430176674028494327449923011041703991733877197660295437827363809989
( 1 / 20 ):	 5 % :	 4891 158 b'styigoa uljjmmd dibiajx fvykz hopyyh gxbrwlh wdjvuwoz lmfmxmp hvnbow ihkgxrn namcagb jxucly sdvygf gnqdmesf uzpajjt vhjvcwb eeaiv mbwnjd ygkntkq rzqbee wfsladkj wnrsrc uxpohkc dibaqef mqpuwm lncqfxk wzkfm xfygkcs wtbqfuf xyxhwv egnpi zmrxk drkjwi kzkhpz kuflbhtm abohleqq iisuna ltbneha kynfy wckbnpn dvwcye lsqoppeu chuezf yixecmn stgwwa zveor sogyh rjrgw tfufugu gauvmo uzvnkavs qazyo pfxeqzxa crwlinhn qskkn csqiyzdi vdfemclx ppnhhbg bsqonc fyyoq qzuaens wytlcmiz hsunga youzyta yjqax qccqlaud nsgaqy nidumvhe mrhidn krxwai rsxhgacm rerng nsinr hxxcao kypdhdbp uwusc xeuelnf whulsyh dhkrqeja jmaawnt uowjxkra zkcfbuhq qidblmg aihva zwnewmg cogsga qynkzfr swumx ddwop dhxxzbd bwnwuwf unhorgqv eobbxeuq qdptymqu jeapfu carlw esm

In [88]:
string.printable.strip()

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [82]:
# 18 April 2024
# testing encryption/decryption 3.1
debug = False

cr = crypto()
pk = cr.generate_private_key()
pubk = cr.private_to_public(pk)
print(pubk)
print(pubk[0])

message_to_encrypt = "hello world hello! hello world hello! hello world hello! hello world hello! hello world hello! hello world hello!"

encrypted_points = cr.encrypt3_1(message_to_encrypt,pubk,1)
print("\n\nencrypted:",encrypted_points)
print("\n\n########################in transit ####################\n",encrypted_points)
decrypted_msg = cr.decrypt3_1(encrypted_points,pk)
print("\n\ndecrypted:",decrypted_msg)

print("\ncompare:before :",message_to_encrypt)
print("compare:after  :",decrypted_msg)
print("compare:isEqual:",decrypted_msg==message_to_encrypt)


(99338537675425655208372115737380253682116498211776055143481655657091070907116, 81793052280019888261651943009892014773138939307884512285270797531068418712981)
99338537675425655208372115737380253682116498211776055143481655657091070907116
113 4 b'hello world hello! hello world hello! hello world hello! hello world hello! hello world hello! hello world hello!'
enc_p1: (65340623205760204254560109147552078071354735744390096175821092331849333462120, 61475805469275661763465933592055441772217920263377960220800772357275755236591)
enc_p1 validity: True


encrypted: b'\x01\x90u{n\xd8u\xf8 \xf9A]\x9a<#\xeb\xb5%R\rh\xa3\xc03\x1c\xb4\xcc1\xc4\xbc\r\xd4h\x03\xd8M\xf0\xc5\xd8\xad\xbd\x1f\xa7\x02\x1a\x8d\xaf\x1a \xa0\x9c.\xbd\x1a\xd9\x95\x81\xb4\xd5%37\xbd[\xf1_\x010`\xd2\xf0\x0e\x15\xb7\xa0\xb9\xc4C>$\xabi\x10,H[\xb4\x18\xa6`\x17r\xbb\x0b\xc1\x16\x9b\x81\xf2\x00\x19\x9e-\x8f\xae\xed\xca\xf9\xbf\x8b?|--\xc3\x15\xc1\xbf0\xb2\x95\xffsU\xd5v%g&\xe2\x84\xab\x03~d@\xc6R\xd1X\xe0\xe7(_\no\x8a\xad\xa2\xc7}b\x

In [71]:
# 17 April 2024
# testing encryption/decryption 3.0

cr = crypto()
pk = cr.generate_private_key()
pubk = cr.private_to_public(pk)
print(pubk)
print(pubk[0])

message_to_encrypt = "hello world hello! hello world hello! hello world hello! hello world hello! hello world hello! hello world hello!"

encrypted_points = cr.encrypt3_0(message_to_encrypt,pubk,1)
print("\n\nencrypted:",encrypted_points)
print("\n\n########################in transit ####################\n",encrypted_points)
decrypted_msg = cr.decrypt3_0(encrypted_points,pk)
print("\n\ndecrypted:",decrypted_msg)

print("\ncompare:before :",message_to_encrypt)
print("compare:after  :",decrypted_msg)
print("compare:isEqual:",decrypted_msg==message_to_encrypt)

  # print("###\nUsing second point")
  # encrypted_points = cr.encrypt2_0("hello world hello"+str(i),pubk,0)
  # print("encrypted:",encrypted_points)
  # print("\n\n########################in transit ####################\n",encrypted_points)
  # decrypted_msg = cr.decrypt2_0(encrypted_points,pk)
  # print("decrypted:",decrypted_msg)

(30702835030431496795203188842056695518914974122610266048273458618132968938655, 18092279904048506828045400321232995566190536762010013984771378816723282987025)
30702835030431496795203188842056695518914974122610266048273458618132968938655
113 4 b'hello world hello! hello world hello! hello world hello! hello world hello! hello world hello! hello world hello!'
enc_p1 validity: True


encrypted: ((43865457896030064412083178193881979143388941952138020960610886661172010497812, 19853582092281144716052701849353825084257659962791678263834822181545667741994), b"\x03\xfcR\xc0y\x87\x0f\xd4\xef\xef\xce\xfc\x0f\x1a[?\x8f\xa4 \xa2\xe2\x07\xb4\x8b\xdc\xba\x10\xa1\x8b\xdcD?\xe1\x00\x15.@\x12\xd1\xbe\xe0\x1e\x00\tG\x1a\xe0\xfd*9\x11\x07\x91x!h\xf4^tSG'\x06\xf6\x994\x01S\xf6$\xa4\xd2\xab\xcb\xa1\x92&\x13\xef\xbf\x19f\xd1H\x12\xad\xbe\x01nd\x11\x0fZ%\xbbW\x96\xb5\xea\x03\xeek\xf7\x1c\x1e\xec\x1e`t\xd1\t(5\x03N\xfd\xba\xd2\\#\xdb\xefz\x88\xbb$V\xc8\x14\xb1LA")


########################in transit #########

In [ ]:
# 15 April 2024
# testing encryption/decryption 2.0

cr = crypto()
pk = cr.generate_private_key()
pubk = cr.private_to_public(pk)
print(pubk)
print(pubk[0])

for i in range(10):
  print("###",i,"\nUsing First point")
  encrypted_points = cr.encrypt2_0("hello world hello"+str(i),pubk,1)
  print("encrypted:",encrypted_points)
  print("\n\n########################in transit ####################\n",encrypted_points)
  decrypted_msg = cr.decrypt2_0(encrypted_points,pk)
  print("decrypted:",decrypted_msg)

  # print("###\nUsing second point")
  # encrypted_points = cr.encrypt2_0("hello world hello"+str(i),pubk,0)
  # print("encrypted:",encrypted_points)
  # print("\n\n########################in transit ####################\n",encrypted_points)
  # decrypted_msg = cr.decrypt2_0(encrypted_points,pk)
  # print("decrypted:",decrypted_msg)

In [ ]:
cr = crypto()
for i in range(1,25):
  print(i,":",cr.tonelli_shanks(i,345345346))

In [ ]:
# 15 April 2024
# testing substraction

cr = crypto()


# print("pubk:",pubk)
# genY = cr.ECxToy(pubk[0])
# print("genY:",genY)
# n_y = (cr.P-pubk[1])%cr.P
# print("n_y:",n_y)
# point = cr.ECadd(pubk[0],pubk[1],pubk[0],n_y)
# print("point:",point)
cnt=0
neg_cnt=0

for i in range(100):
  cnt+=1
  pk = cr.generate_private_key()
  p1 = cr.private_to_public(pk)

  pk2 = cr.generate_private_key()
  p2 = cr.private_to_public(pk2)

  p3 = cr.ECadd(p1[0],p1[1],p2[0],p2[1])

  p4 = cr.ECsubtract(p3[0],p3[1],p2[0],p2[1])
  # print("p1:",p1)
  # print("p2:",p2)
  # print("p3:",p3)
  # print("p4:",p4)
  #print(p1==p4)
  if(p1!=p4):
    neg_cnt+=1

print("cnt:",cnt," neg_cnt:",neg_cnt)


#Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240  # x co-ordinate of generating point of secp256k1 i.e. curve used in bitcoin
#Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424  # y co-ordinate of generating point of secp256k1 i.e. curve used in bitcoin



cnt: 100  neg_cnt: 0


In [ ]:
P = 2 ** 256 - 2 ** 32 - 2 ** 9 - 2 ** 8 - 2 ** 7 - 2 ** 6 - 2 ** 4 - 1
x = (P+1)//4
y= (P+1)/4
print(x)
print(y)

28948022309329048855892746252171976963317496166410141009864396001977208667916
2.894802230932905e+76


In [ ]:
#test y from x
cr = crypto()
pk = cr.generate_private_key()
pubk = cr.private_to_public(pk)
pubk2 = cr.private_to_public(cr.order-pk)
print(pubk)
print(pubk2)
print(pubk[0])
generated_y = cr.ECxToy(pubk[0])
print(generated_y)

#generated_y = cr.ECxToyNotWorking(pubk[0])
#print(generated_y)


(56319530231976224183438377473195172496306287508066867137288889090984035455602, 88592662805159883298140369776025028159328430640934484661820602453851464617457)
(56319530231976224183438377473195172496306287508066867137288889090984035455602, 27199426432156312125430615232662879693941554024706079377636981554057370054206)
56319530231976224183438377473195172496306287508066867137288889090984035455602
(88592662805159883298140369776025028159328430640934484661820602453851464617457, 27199426432156312125430615232662879693941554024706079377636981554057370054206)


In [ ]:
cr = crypto()
cr.P


115792089237316195423570985008687907853269984665640564039457584007908834671663

In [ ]:
cr.Gy

In [ ]:
pk = 0x6385BD34097903323E69A55B32E383C2EB13FFA984222A61D84CDE8DE4703962
pubx,puby = cr.private_to_public(pk)
print(hex(pubx))
print(hex(puby))



0x2a7e7ea4bac6e3c384be6040a030674c78685e2c31fb6f32aab26ebd49ebb951
0xeef5384db9bfee5faf530fca4d2ab8e5b426a6629fcdbddda1c91e830e6197b6


In [ ]:
msg = 0x00C1D0B270F6AC2D9CDC4FC1105D6F1D993D1F2E845FB6F89C705EC6DEB2B3
rn = 0xE7A48DC49D7FE9059E47CBE390E1163EEA5796B4A6AAC77736E9ECFA8500AE48
sign = cr.sign_message(msg,rn,pk)
print(sign)

signing....
rx: 99115877571138904525161819566697840382180511416650522780856252060860694793420
sign: 60068741763705915393223465090797128619537592644105160517755440237969953417130
(99115877571138904525161819566697840382180511416650522780856252060860694793420, 60068741763705915393223465090797128619537592644105160517755440237969953417130)


In [ ]:
print(cr.verify_message(sign[1],sign[0],msg,pubx,puby))

verifying...
m_s: 7654833023648714401928392016443769895283393743334358284275130549657300713275
r_s: 34726424671263813174707202711466381231902374951939214054980053334720705716854
(x,y): (92685075176497724014940311779812843069141506006760926978332164669386080918818, 43942772451509575587469516647184302101407859412812430308898174899807860982857)
(x2,y2): (95412658106290389209996840386726041469899777348374304569468322805084038779545, 17475077123532806769864238591713645415292695891791488591048210907938569122254)
(x3,y3): (99115877571138904525161819566697840382180511416650522780856252060860694793420, 76897929825134835200573813543687982047400288155864558694013583111222351156740)
True


In [ ]:
int(0x4CB32FFDA3EFA569CE7CFD3F9FCFC8D0849AC3015EAA87D430A29D9CA7F0D6)

135517083348175275410893551749236660304032912305017035934790592838100316374